In [1]:
!pip install selenium

     |████████████████████████████████| 904 kB 5.4 MB/s eta 0:00:01


In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://store.steampowered.com/search/?filter=topsellers'

user_agent = {'User-agent': 'Mozilla/5.0'}
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [8]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [5]:
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]

In [6]:
len(rows)

50

In [13]:
#this was sufficient to scroll to the bottom of the top sellers page for steam, pulling code from scraping list
for i in range(500):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    time.sleep(1)

In [15]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [16]:
#successfully scraped 16.5k elements, hopefully all games.
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]
rows[0]
len(rows)

16483

In [32]:
#pulling code created in 'scraping_list' notebook to create dataframe.
#dataframe index is game name, 

games = {}

for row in rows:
    title = (row.find('span', class_='title').text)
    url = row.get('href')
    release_date = (row.find('div', class_='col search_released responsive_secondrow').text)
    if (row.find('div', class_='col search_discount responsive_secondrow').text.strip()) == '':
        been_sale = 0
    else:
        been_sale = 1
    if been_sale == 0:
        orig_price = (row.find('div', class_='col search_price responsive_secondrow').text.strip())
    else:
        orig_price = (row.find('strike').text)
    games[title] = (url, 
                    release_date,
                    been_sale,
                    orig_price)
    

In [34]:
gamesdf1 = pd.DataFrame(games).T
gamesdf1.columns = ['link', 'release_date', 'been_sale', 'orig_price']

gamesdf1.shape

(16374, 4)

In [35]:
#no nulls!!
gamesdf1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16374 entries, OUTRIDERS to RPG Maker MZ Bundle S
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          16374 non-null  object
 1   release_date  16374 non-null  object
 2   been_sale     16374 non-null  object
 3   orig_price    16374 non-null  object
dtypes: object(4)
memory usage: 639.6+ KB


In [36]:
gamesdf1.head()

,link,release_date,been_sale,orig_price
OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99
Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99
Halo: The Master Chief Collection,https://store.steampowered.com/sub/376282/?snr...,"Dec 3, 2019",1,$39.99
It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99
The Legend of Heroes: Trails of Cold Steel IV Digital Deluxe Edition (Launch Week only),https://store.steampowered.com/bundle/20467/Th...,,0,$79.96


In [37]:
gamesdf1.tail()

,link,release_date,been_sale,orig_price
Kai Entity,https://store.steampowered.com/app/931620/Kai_...,"Sep 18, 2018",1,$0.99
aMAZE Dark Times,https://store.steampowered.com/app/692200/aMAZ...,"Aug 17, 2017",1,$0.99
Command Modern Operations + Tacview Advanced,https://store.steampowered.com/bundle/12584/Co...,,0,$112.48
Last Year Ultimate Edition,https://store.steampowered.com/bundle/16734/La...,,0,$20.21
RPG Maker MZ Bundle S,https://store.steampowered.com/bundle/15854/RP...,,0,$158.89
